# Research Project - Google Colab版

このノートブックは研究プロジェクトをGoogle Colabで実行するためのものです。

## 1. 環境セットアップ

In [ ]:
# Colabかどうかの確認
try:
    import google.colab
    IN_COLAB = True
    print("✅ Google Colab環境で実行中")
except:
    IN_COLAB = False
    print("❌ ローカル環境で実行中")

In [ ]:
# 必要なライブラリのインストール
if IN_COLAB:
    !pip install -q numpy pandas matplotlib seaborn plotly beautifulsoup4 requests

In [ ]:
# Google Driveのマウント
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
# GitHubからプロジェクトをクローン（必要に応じて）
# !git clone https://github.com/yourusername/research-project.git /content/research-project
# %cd /content/research-project

## 2. プロジェクトファイルのアップロード

In [ ]:
# ローカルファイルのアップロード
if IN_COLAB:
    from google.colab import files
    
    print("プロジェクトファイルをアップロードしてください:")
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        print(f'Uploaded: {filename}')

## 3. メインスクリプトの実行

In [ ]:
# 共通のインポート
import os
import sys
import json
import datetime
import subprocess
from pathlib import Path

In [ ]:
# Colab用の作業ディレクトリ設定
if IN_COLAB:
    WORK_DIR = '/content/research_project'
    if not os.path.exists(WORK_DIR):
        os.makedirs(WORK_DIR)
    os.chdir(WORK_DIR)
else:
    WORK_DIR = os.getcwd()

print(f"作業ディレクトリ: {WORK_DIR}")

## 4. 自動整理・保存システム

In [ ]:
# auto_organize_and_save.pyの内容をColab用に適応
def auto_organize_and_save_colab():
    """Colab環境用の自動整理・保存関数"""
    print("=" * 50)
    print("🔄 自動整理・保存システム (Colab版)")
    print("=" * 50)
    
    # セッションディレクトリの作成
    session_dir = os.path.join(WORK_DIR, 'sessions')
    if not os.path.exists(session_dir):
        os.makedirs(session_dir)
    
    # 現在の日時
    now = datetime.datetime.now()
    date_str = now.strftime('%Y-%m-%d')
    time_str = now.strftime('%H:%M:%S')
    
    # セッションファイル名
    session_file = os.path.join(session_dir, f'AUTO_SESSION_SAVE_{date_str}.md')
    
    # セッション内容の作成
    content = f"""# 🔄 自動セッション保存 - {date_str}

## 📅 実行情報
- **実行時刻**: {time_str}
- **環境**: Google Colab
- **作業ディレクトリ**: {WORK_DIR}

## 📁 ファイル状態
"""
    
    # ファイル一覧の取得
    for root, dirs, files in os.walk(WORK_DIR):
        # sessions, __pycache__などを除外
        dirs[:] = [d for d in dirs if not d.startswith('.') and d not in ['__pycache__', 'sessions']]
        
        level = root.replace(WORK_DIR, '').count(os.sep)
        indent = ' ' * 2 * level
        content += f"{indent}{os.path.basename(root)}/\n"
        
        sub_indent = ' ' * 2 * (level + 1)
        for file in files:
            if not file.startswith('.') and not file.endswith('.pyc'):
                content += f"{sub_indent}{file}\n"
    
    content += "\n## ✅ 完了\n"
    content += f"セッションが {time_str} に保存されました。\n"
    
    # ファイルに保存
    with open(session_file, 'w', encoding='utf-8') as f:
        f.write(content)
    
    print(f"✅ セッションを保存しました: {session_file}")
    
    # Google Driveにも保存（マウントされている場合）
    if IN_COLAB and os.path.exists('/content/drive/MyDrive'):
        drive_backup_dir = '/content/drive/MyDrive/research_backup'
        if not os.path.exists(drive_backup_dir):
            os.makedirs(drive_backup_dir)
        
        drive_session_file = os.path.join(drive_backup_dir, f'AUTO_SESSION_SAVE_{date_str}.md')
        with open(drive_session_file, 'w', encoding='utf-8') as f:
            f.write(content)
        
        print(f"✅ Google Driveにもバックアップしました: {drive_session_file}")

# 実行
auto_organize_and_save_colab()

## 5. Vercel統合システム (Colab版)

In [ ]:
# Vercel CLIのインストール（必要な場合）
if IN_COLAB:
    !npm install -g vercel

In [ ]:
# Vercel設定の確認
def check_vercel_config():
    """Vercel設定をチェック"""
    vercel_json_path = os.path.join(WORK_DIR, 'vercel.json')
    
    if os.path.exists(vercel_json_path):
        with open(vercel_json_path, 'r') as f:
            config = json.load(f)
        print("✅ vercel.json found:")
        print(json.dumps(config, indent=2))
    else:
        print("⚠️ vercel.json not found. Creating default config...")
        default_config = {
            "buildCommand": "echo 'No build required'",
            "outputDirectory": "public",
            "framework": null
        }
        with open(vercel_json_path, 'w') as f:
            json.dump(default_config, f, indent=2)
        print("✅ Created default vercel.json")

check_vercel_config()

## 6. ユーティリティ関数

In [ ]:
# ファイルダウンロード関数
def download_files(file_patterns):
    """指定されたパターンのファイルをダウンロード"""
    if not IN_COLAB:
        print("This function is for Colab only")
        return
    
    from google.colab import files
    import glob
    
    for pattern in file_patterns:
        for file_path in glob.glob(pattern):
            print(f"Downloading: {file_path}")
            files.download(file_path)

# 使用例
# download_files(['*.html', 'public/*.html'])

In [ ]:
# プロジェクトをZIPファイルとしてダウンロード
def download_project_as_zip():
    """プロジェクト全体をZIPファイルとしてダウンロード"""
    if not IN_COLAB:
        print("This function is for Colab only")
        return
    
    import shutil
    from google.colab import files
    
    zip_name = f"research_project_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
    shutil.make_archive(zip_name, 'zip', WORK_DIR)
    
    files.download(f"{zip_name}.zip")
    print(f"✅ Downloaded: {zip_name}.zip")

# 実行する場合はコメントを外してください
# download_project_as_zip()

## 7. 定期実行タスク（Colab Pro用）

In [ ]:
# Colab Proで長時間実行する場合の定期保存
import time
import threading

def periodic_save(interval_minutes=30):
    """定期的に作業を保存"""
    def save_task():
        while True:
            time.sleep(interval_minutes * 60)
            try:
                auto_organize_and_save_colab()
                print(f"⏰ 定期保存完了 ({datetime.datetime.now().strftime('%H:%M:%S')})")
            except Exception as e:
                print(f"❌ 定期保存エラー: {e}")
    
    # バックグラウンドスレッドで実行
    thread = threading.Thread(target=save_task, daemon=True)
    thread.start()
    print(f"✅ 定期保存を開始しました（{interval_minutes}分ごと）")

# 定期保存を有効にする場合はコメントを外してください
# periodic_save(30)

## 8. プロジェクト固有のコード実行

ここに研究プロジェクト固有のコードを追加してください。

In [ ]:
# プロジェクト固有のコードをここに追加
# 例: データ分析、可視化、モデル学習など